<a href="https://colab.research.google.com/github/varsha108/python-programs/blob/master/Chatbot_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
data = {"intents": [
    {"tag":"greeting",
     "patterns":["Hi", "Hello", "How are you doing?", "Wassup?", "Yo"],
     "responses": ["Howdy!", "Yo!", "Hello there!", "Hiya"]
     },
    {"tag":"age",
     "patterns":["How old are you?", "What is your age?", "When were you born?"],
     "responses": ["I am 23", "I was born in 2000", "I'm an adult, aged 23", "My birth day is 4th June, and year 2000"]
     },
    {"tag":"daily",
     "patterns":["what do you do?", "Do you like hanging out?", "How is your day?"],
     "responses": ["I am a student and I am mostly free on weekends", "My day goes by studing, going to classes and hanging out", "I do like to hangout on weekends"]
    },
    {"tag":"name",
     "patterns":["What is your name?", "What do I call you?", "Can I know your name?"],
     "responses": ["My name is Shiro", "I'm Shiro, nice to make your acquaintance!", "I'm known as Shiro"]
     },
    {"tag":"Hobby",
     "patterns":["What do like to do in your freetime?", "What is your hobby?", "How do you spend free time?"],
     "responses": ["I like to read books. My favourite genre is crime thriller.", "I'm a book worm", "I spend my time reading fiction novels"]
     },
    {"tag":"goodbye",
     "patterns":["bye","got to go","see ya","adios","cya","ciao"],
     "responses":["It was nice speaking to you","See you later","Speak Soon", "Bubye"]
        },
 ]}
import warnings
warnings.filterwarnings("ignore")

import json
import string
import random

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")
#Lemmatization
lemmatizer = WordNetLemmatizer()
words =[]
classes=[]
doc_x=[]
doc_y=[]
#Tokenization
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    doc_x.append(pattern)
    doc_y.append(intent["tag"])
  if intent["tag"] not in classes:
      classes.append(intent["tag"])
#Converting to lowercase
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
#Sorting in alphabetic order and removing redundancy
words = sorted(set(words))
classes = sorted(set(classes))
#Training data list
training = []
out_empty = [0]*len(classes)
#Creating Bag of Words
for idx, doc in enumerate(doc_x):
    bow=[]
    text=lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row=list(out_empty)
    output_row[classes.index(doc_y[idx])]=1

    training.append([bow, output_row])

random.shuffle(training)
train = np.array(training, dtype = object)
train_x = np.array(list(train[:,0]))
train_y = np.array(list(train[:,1]))
#Model for prediction
input_shape = (len(train_x[0]),)
output_shape = len(train_y[0])
epochs = 500
#Creating sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation ='relu'))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = 'softmax'))
#Creating ADAM (Adaptive moment estimation) optimizer with a given learning rate
adam = tf.keras.optimizers.Adam(learning_rate = 0.01)
#Compiling training model using ADAM
model.compile(loss='categorical_crossentropy',
              optimizer = adam,
              metrics =['accuracy'])
print (model.summary())
#Fitting the model
model.fit(x=train_x, y=train_y, epochs=500, verbose=1)
#Defining function response to generate reply to user input
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0]*len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word==w:
        bow[idx]=1
  return np.array(bow)
def predict_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  output = model.predict(np.array([bow]))[0]
  threshold = 0.2
  y_predict = [[idx, res] for idx, res in enumerate(output) if res>threshold]

  y_predict.sort(key = lambda x:x[1], reverse = True)
  output_list = []
  for r in y_predict:
    output_list.append(labels[r[0]])
  return output_list

def get_response (intents_list, intents_json):
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
    return result
#Running the Chatbot
while True:
  message = input("")
  intents = predict_class(message,words,classes)
  result = get_response(intents,data)
  print (result)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               5376      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 390       
                                                                 
Total params: 14022 (54.77 KB)
Trainable params: 14022 (54.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


None
Epoch 1/500
1/1 [==============================] - 1s 815ms/step - loss: 1.7869 - accuracy: 0.3478
Epoch 2/500
1/1 [==============================] - 0s 13ms/step - loss: 1.7630 - accuracy: 0.2609
Epoch 3/500
1/1 [==============================] - 0s 12ms/step - loss: 1.5854 - accuracy: 0.4348
Epoch 4/500
1/1 [==============================] - 0s 10ms/step - loss: 1.5792 - accuracy: 0.4348
Epoch 5/500
1/1 [==============================] - 0s 10ms/step - loss: 1.4515 - accuracy: 0.5217
Epoch 6/500
1/1 [==============================] - 0s 11ms/step - loss: 1.3184 - accuracy: 0.6087
Epoch 7/500
1/1 [==============================] - 0s 10ms/step - loss: 1.2737 - accuracy: 0.6522
Epoch 8/500
1/1 [==============================] - 0s 9ms/step - loss: 1.0691 - accuracy: 0.6957
Epoch 9/500
1/1 [==============================] - 0s 10ms/step - loss: 1.0004 - accuracy: 0.6957
Epoch 10/500
1/1 [==============================] - 0s 11ms/step - loss: 0.8215 - accuracy: 0.7391
Epoch 11/500
1

KeyboardInterrupt: ignored